In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the datetime module from the datetime library.
from datetime import datetime

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

## Create a set of 2000 random latitudes and longitudes.

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
# Output is a zip object in memory
coordinates = zip(lats, lngs)
coordinates

## Get the nearest city using the citipy module.

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

750

## Perform an API call with the OpenWeatherMap.

In [4]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

## Retrieve the following from the API Call:
- Latitude & Longitude
- Maximum Temperature
- Percent Humidity
- Percent Cloudiness
- Wind Speed
- Weather description

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather['wind']['speed']
        city_description = city_weather['weather']['description']
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind": city_wind,
                          "Description": city_description,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
City not found. Skipping...
Processing Record 2 of Set 1 | tecoanapa
City not found. Skipping...
Processing Record 3 of Set 1 | cartagena del chaira
City not found. Skipping...
Processing Record 4 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 5 of Set 1 | tiksi
City not found. Skipping...
Processing Record 6 of Set 1 | bethel
City not found. Skipping...
Processing Record 7 of Set 1 | lensk
City not found. Skipping...
Processing Record 8 of Set 1 | ituni
City not found. Skipping...
Processing Record 9 of Set 1 | puerto ayora
City not found. Skipping...
Processing Record 10 of Set 1 | tokur
City not found. Skipping...
Processing Record 11 of Set 1 | rawson
City not found. Skipping...
Processing Record 12 of Set 1 | mataura
City not found. Skipping...
Processing Record 13 of Set 1 | zhoucheng
City not found. Skipping...
Processing Record 14 of Set 1 | port elizabeth
City

City not found. Skipping...
Processing Record 20 of Set 3 | new norfolk
City not found. Skipping...
Processing Record 21 of Set 3 | baglung
City not found. Skipping...
Processing Record 22 of Set 3 | castro
City not found. Skipping...
Processing Record 23 of Set 3 | illoqqortoormiut
City not found. Skipping...
Processing Record 24 of Set 3 | altona
City not found. Skipping...
Processing Record 25 of Set 3 | caravelas
City not found. Skipping...
Processing Record 26 of Set 3 | yakeshi
City not found. Skipping...
Processing Record 27 of Set 3 | kayes
City not found. Skipping...
Processing Record 28 of Set 3 | umm lajj
City not found. Skipping...
Processing Record 29 of Set 3 | banff
City not found. Skipping...
Processing Record 30 of Set 3 | gafanha da encarnacao
City not found. Skipping...
Processing Record 31 of Set 3 | rapid valley
City not found. Skipping...
Processing Record 32 of Set 3 | half moon bay
City not found. Skipping...
Processing Record 33 of Set 3 | palembang
City not fo

City not found. Skipping...
Processing Record 39 of Set 5 | liverpool
City not found. Skipping...
Processing Record 40 of Set 5 | carutapera
City not found. Skipping...
Processing Record 41 of Set 5 | severo-kurilsk
City not found. Skipping...
Processing Record 42 of Set 5 | solnechnyy
City not found. Skipping...
Processing Record 43 of Set 5 | moron
City not found. Skipping...
Processing Record 44 of Set 5 | dukat
City not found. Skipping...
Processing Record 45 of Set 5 | yushu
City not found. Skipping...
Processing Record 46 of Set 5 | guerrero negro
City not found. Skipping...
Processing Record 47 of Set 5 | vaini
City not found. Skipping...
Processing Record 48 of Set 5 | gimbi
City not found. Skipping...
Processing Record 49 of Set 5 | kirkwall
City not found. Skipping...
Processing Record 50 of Set 5 | zhicheng
City not found. Skipping...
Processing Record 1 of Set 6 | doka
City not found. Skipping...
Processing Record 2 of Set 6 | iqaluit
City not found. Skipping...
Processing 

In [6]:
len(city_data)

692

## Add the data to a new DataFrame.

In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tasiilaq,65.6145,-37.6368,48.36,97,100,12.97,GL,2021-07-11 12:06:08
1,Hasaki,35.7333,140.8333,70.29,93,19,17.00,JP,2021-07-11 12:06:08
2,Cobija,-11.0267,-68.7692,73.69,70,84,1.28,BO,2021-07-11 12:06:09
3,Norman Wells,65.2820,-126.8329,56.80,71,98,0.96,CA,2021-07-11 12:06:09
4,Busselton,-33.6500,115.3333,56.05,67,93,17.38,AU,2021-07-11 12:06:09
5,Hobart,-42.8794,147.3294,42.26,87,0,4.00,AU,2021-07-11 12:06:09
6,Rodrigues Alves,-7.7380,-72.6509,70.09,99,100,0.47,BR,2021-07-11 12:06:10
7,Punta Arenas,-53.1500,-70.9167,32.11,62,68,22.15,CL,2021-07-11 12:01:43
8,Milledgeville,33.0801,-83.2321,73.51,96,90,0.00,US,2021-07-11 12:06:10
9,Mataura,-46.1927,168.8643,39.67,96,99,3.27,NZ,2021-07-11 12:06:11


## Export the DataFrame as a csv file and save it as WeatherPy_Database.csv

In [ ]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")